In [1]:
# Read CSV from OneLake (Fabric)
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(
        "abfss://94910407-0b7b-4043-96d7-ad567374a8fd@onelake.dfs.fabric.microsoft.com/9ef8c1c9-dd44-4787-b11a-608415ca72fc/Files/GlobalLandTemperaturesByCity_ENHANCED_v2.csv"
    )

# Save raw data as Delta table in Lakehouse
df.write \
  .mode("overwrite") \
  .format("delta") \
  .saveAsTable("raw_global_land_temperatures")


StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 3, Finished, Available, Finished)

In [2]:
# Check if data is loaded
df_check = spark.read.table("raw_global_land_temperatures")

# Show sample records
df_check.show(5)

# Check number of rows
df_check.count()

# Check schema
df_check.printSchema()


StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 4, Finished, Available, Finished)

+----------+------------------+-----------------------------+---------+------------+---------+--------+---------+--------+---------+-------------+----+-----+------+
|      Date|AverageTemperature|AverageTemperatureUncertainty|     City|     Country|   Region|Latitude|Longitude|Humidity|WindSpeed|Precipitation|Year|Month|Season|
+----------+------------------+-----------------------------+---------+------------+---------+--------+---------+--------+---------+-------------+----+-----+------+
|10/05/1992|             27.92|                         0.64|   Sydney|   Australia|Australia|  33.86S|  151.21E|    56.7|     1.95|          0.0|1992|    5|Spring|
|03/06/2020|             26.24|                         0.27|    Delhi|       India|     Asia|  28.70N|   77.10E|    48.3|     8.11|         0.37|2020|    6|Summer|
|23/09/2013|              6.04|                         0.36|Cape Town|South Africa|   Africa|  33.92S|   18.42E|    52.0|     7.11|         4.57|2013|    9|Autumn|
|15/03/199

In [3]:
from pyspark.sql.functions import col, avg

# 1) Read raw table
df = spark.read.table("raw_global_land_temperatures")

# 2) Clean data
df_clean = (
    df.dropDuplicates()
      .na.drop(subset=["Date", "AverageTemperature", "City", "Country"])
      .withColumn("Date", col("Date").cast("date"))
      .withColumn("AverageTemperature", col("AverageTemperature").cast("double"))
)

# 3) Save cleaned detailed table
df_clean.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("clean_global_land_temperatures")

# 4) Simple final table: avg temp per country and year
df_final = (
    df_clean
      .groupBy("Country", "Year")
      .agg(avg("AverageTemperature").alias("avg_temperature"))
)

df_final.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("final_avg_temp_country_year")


StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 5, Finished, Available, Finished)

In [4]:
# Store cleaned dataset as Delta table in Fabric Lakehouse

df_clean.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("clean_global_land_temperatures")


StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 6, Finished, Available, Finished)

In [5]:
%%sql
-- SQL Query 1: Average temperature by country
SELECT
    Country,
    AVG(AverageTemperature) AS avg_temperature
FROM clean_global_land_temperatures
GROUP BY Country
ORDER BY avg_temperature DESC;

StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 7, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 2 fields>

In [6]:
%%sql
-- SQL Query 2: Average temperature by year
SELECT
    Year,
    AVG(AverageTemperature) AS avg_temperature
FROM clean_global_land_temperatures
GROUP BY Year
ORDER BY Year;

StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 8, Finished, Available, Finished)

<Spark SQL result set with 31 rows and 2 fields>

In [7]:
# 1) Load the cleaned table
df = spark.read.table("clean_global_land_temperatures")

# 2) Look at a few rows
df.show(10)

# 3) See the schema (column names and types)
df.printSchema()

# 4) Basic stats for numeric columns
df.describe().show()

# 5) How many rows and columns?
row_count = df.count()
col_count = len(df.columns)
print("Rows:", row_count, " Columns:", col_count)

# 6) List some countries
df.select("Country").distinct().show(20)

# 7) Average temperature by country (quick check)
from pyspark.sql.functions import avg
df.groupBy("Country").agg(avg("AverageTemperature").alias("avg_temp")).show(20)


StatementMeta(, a50fd072-d91d-489f-9acc-727a4342e4b0, 9, Finished, Available, Finished)

+----+------------------+-----------------------------+-----------+-------------+-------------+--------+---------+--------+---------+-------------+----+-----+------+
|Date|AverageTemperature|AverageTemperatureUncertainty|       City|      Country|       Region|Latitude|Longitude|Humidity|WindSpeed|Precipitation|Year|Month|Season|
+----+------------------+-----------------------------+-----------+-------------+-------------+--------+---------+--------+---------+-------------+----+-----+------+
|NULL|             22.48|                         0.78|Los Angeles|United States|North America|  34.05N|  118.24W|    62.8|     3.11|         3.92|2019|    4|Spring|
|NULL|             20.18|                         0.61|  Cape Town| South Africa|       Africa|  33.92S|   18.42E|    55.7|     5.23|         0.36|1997|    8|Summer|
|NULL|             15.82|                         0.67|Los Angeles|United States|North America|  34.05N|  118.24W|    63.4|    11.21|          0.0|2007|    1|Winter|
|NUL